In [1]:
import header
from header import __root__

# Internal modules
from src import gs

🔑 Found password in password.txt (DEBUG MODE)
✅ Successfully opened KeePass database: C:\Users\user\Documents\repos\hypotez\secrets\credentials.kdbx
Failed to load GAPI credentials


In [2]:
import importlib
import os
import asyncio
import time
from pathlib import Path
from types import SimpleNamespace
from typing import Optional, List, Any
from dataclasses import dataclass, field


from src.suppliers.suppliers_list import *
from src.suppliers.get_graber_by_supplier  import get_graber_by_supplier_prefix, get_graber_by_supplier_url
from src.suppliers.graber import Graber
from src.webdriver.driver import Driver
from src.webdriver.firefox import Firefox
from src.webdriver.chrome import Chrome
from src.llm.gemini import GoogleGenerativeAi
from src.llm.openai.model import OpenAIModel
from src.endpoints.prestashop.product import PrestaProduct
from src.endpoints.prestashop.language import PrestaLanguage
from src.endpoints.prestashop.product_fields import ProductFields
from src.endpoints.advertisement.facebook.scenarios.post_message import (
    post_message,
)
from src.utils.file import read_text_file, save_text_file, get_filenames_from_directory

from src.utils.jjson import j_loads, j_loads_ns, j_dumps
from src.utils.image import get_image_bytes, get_raw_image_data
from src.logger.logger import logger

2025-05-23 23:57:25,134 - INFO - Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [3]:
# --- file config.py
class Config:


    ENDPOINT: Path = __root__ /'src' / 'endpoints' / 'emil'
    SUPPLIERS_ENDPOINT: Path = __root__ / 'src' / 'suppliers' / 'suppliers_list'
    config:SimpleNamespace = j_loads_ns(ENDPOINT / 'emil.json')
    GEMINI_API_KEY:str = gs.credentials.gemini.onela.api_key
    PRESTA_API_KEY:str = gs.credentials.prestashop.store_davidka_net.api_key
    PRESTA_DOMAIN:str = gs.credentials.prestashop.store_davidka_net.api_domain
    gemini_model_name:str = config.gemini_model_name
    system_instruction:str = ' ' # <- Это пробел!
    webdriver_window_mode:str = 'headless'
# --- end file config.pt

In [4]:
class ScenarioPipeline:
    """Dataclass for designing and promoting images through various platforms."""

    gemini: Optional[GoogleGenerativeAi] = None
    openai: Optional[OpenAIModel] = None
    product: PrestaProduct = None
    driver: Driver = None

    def __init__(self,
            presta_api_key:Optional[str] = '',
            presta_api_domain:Optional[str] = '',
            gemini_model_name:Optional[str] = '',
            openai_model_name:Optional[str] = '',
            gemini_api_key:Optional[str] = '',
            openai_api_key:Optional[str] = '',
            gemini: Optional[GoogleGenerativeAi] = None, 
            openai: Optional[OpenAIModel] = None,
            system_instruction:str = '',
            driver:Driver = None, 
            webdriver_window_mode:str = ''
            ):
        """
        Инициализация 
            Args:
                presta_api_key:Optional[str] = '',
                presta_api_domain:Optional[str] = '',
                gemini_model_name:Optional[str] = '',
                openai_model_name:Optional[str] = '',
                gemini_api_key:Optional[str] = '',
                openai_api_key:Optional[str] = '',
                gemini: Optional[GoogleGenerativeAi] = None, 
                openai: Optional[OpenAIModel] = None,
        """
        ...
        if driver:
            self.driver = driver
        else:
            self.driver = Driver(Firefox, 
                                 window_mode=webdriver_window_mode if webdriver_window_mode else Config.webdriver_window_mode,
                                )

        if gemini:
            self.gemini = gemini
        else:
            gemini_api_key:str = gemini_api_key if gemini_api_key else Config.GEMINI_API_KEY
            gemini_model_name:str = gemini_model_name if gemini_model_name else Config.gemini_model_name
            system_instruction:str = system_instruction if system_instruction else Config.system_instruction
            if not self._init_gemini(gemini_api_key, gemini_model_name, system_instruction):
                logger.debug('Модель GEMINI не иницаилизирована')

        presta_api_key:str = presta_api_key if presta_api_key else Config.PRESTA_API_KEY
        presta_api_domain:str = presta_api_domain if presta_api_domain else Config.PRESTA_DOMAIN
        if not presta_api_key or not presta_api_domain:
            logger.critical(f'Проверь \nAPI {presta_api_key}\nDomain {presta_api_domain=}')
            return False

        self.product = PrestaProduct(presta_api_key, presta_api_domain )

    def _init_gemini(self, api_key: str, model_name: str, system_instruction: str) -> bool:
        """"""
        try:
            generation_config = dict({'response_mime_type':'application/json'})
            self.gemini = GoogleGenerativeAi(api_key, model_name, generation_config, system_instruction)
            return True
        except Exception as ex:
            logger.error(f'Ошибка иницализации модели!', ex, False)
            return False


    async def process_supplier(self, supplier_prefix:str) -> bool:
        """"""
        ...
        try:
            supplier_path:Path = Config.SUPPLIERS_ENDPOINT / supplier_prefix 
            graber: Graber = get_graber_by_supplier_prefix(self.driver, supplier_prefix)
            scenarios_list: list[dict] = j_loads(Config.SUPPLIERS_ENDPOINT / supplier_prefix / 'scenarios')
            locators_path:Path = supplier_path / 'locators' 
            locator_product:SimpleNamespace = j_loads_ns(locators_path / 'product.json')
            locator_category:SimpleNamespace = j_loads_ns(locators_path / 'category.json')
            categories_crawler:Any = None
            categories_crawler_module_path:str = f"src.suppliers.suppliers_list.{supplier_prefix}.categories_crawler"
        except Exception as ex:
            logger.error(f'Непредвиденная ошибка', ex)
            return False

        try:
            categories_crawler = importlib.import_module(categories_crawler_module_path)
        except Exception as ex:
            logger.error(f"Failed to import module `categories_crawler` '{supplier_prefix}'", ex)
            return False
        
        for scenario in scenarios_list:
            self.driver.get_url(scenario.url)

            products_urls_in_category:list = categories_crawler.get_list_products_in_category(self.driver, locator_category)
            for product_url in products_urls_in_category:
                self.driver.get_url(product_url)
                product_fields:ProductFields = await graber.grab_page_async()
                ...

            
    async def process_suppliers_list(self, suppliers_prefixes: str|list) -> bool:
        """
        Process suppliers based on the provided prefix.
        Args:
            suppliers_prefixes (Optional[str | List[str, str]], optional): Prefix for suppliers. Defaults to ''.
        Returns:
            bool: True if processing is successful, False otherwise.
        Raises:
            Exception: If any error occurs during supplier processing.
        """
        
        for supplier_prefix in suppliers_prefixes:
            try:
                await self.process_supplier(supplier_prefix)
            except Exception as ex:
                logger.error(f'Error while processing suppliers: {ex}')
                continue

In [5]:
driver:Driver = None

try:
    driver = Driver(Firefox, window_mode = 'headless')
except Exception as ex:
    logger.critical(f'Ошибка инициализации шебдрайвера: ', ех, False)  

2025-05-23 23:57:25,342 - INFO - ℹ️ Инициализация Firefox WebDriver 
2025-05-23 23:57:25,342 - DEBUG - 🐛 Текущий __root__: C:\Users\user\Documents\repos\hypotez 
NoneType: None
2025-05-23 23:57:25,350 - DEBUG - 🐛 Конфигурация загружена. enable_geckodriver_log: True 
NoneType: None
2025-05-23 23:57:25,351 - INFO - ℹ️ Попытка настроить логирование geckodriver... 
2025-05-23 23:57:25,353 - DEBUG - 🐛 Предполагаемый путь к лог-файлу geckodriver: C:\Users\user\Documents\repos\hypotez\geckodriver.log 
NoneType: None
2025-05-23 23:57:25,354 - INFO - ℹ️ Логирование geckodriver настроено. Путь к лог-файлу: C:\Users\user\Documents\repos\hypotez\geckodriver.log 
2025-05-23 23:57:30,130 - INFO - ℹ️ Браузер Firefox успешно запущен. Режим окна: headless. 


In [6]:
executor: ScenarioPipeline = ScenarioPipeline(driver = driver)
suppliers_prefixes_list:list = ['hb']  
        
try:
    await executor.process_suppliers_list(suppliers_prefixes_list)
except Exception as ex:
    logger.error(f'Wxcepation: ', ex, False)
finally:
    driver.quit()
    

2025-05-23 23:57:30,141 - INFO - ℹ️ Модель models/gemini-2.5-flash-preview-04-17 инициализирована 
2025-05-23 23:57:30,753 - ERROR - ❌ Error while processing suppliers: 'dict' object has no attribute 'url' 
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_13752\1953721086.py", line 115, in process_suppliers_list
    await self.process_supplier(supplier_prefix)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_13752\1953721086.py", line 93, in process_supplier
    self.driver.get_url(scenario.url)
                        ^^^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'url'
